Import the main PyTorch package along with the *nn* package which is used to build the model. numpy will be used to pre-process data.

In [1]:
import torch
from torch import nn

import numpy as np

Define a custom dataset of sentences that model should output when fed with the first word or the first few characters.

This is followed by creating a dictionary out of all the characters in the sentences and map them to an integer. This will allow to convert  input characters to their respective integers (*char2int*) and vice versa (*int2char*).

In [4]:
text = ['hey how are you','good i am fine','have a nice day']

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

In [3]:
print(char2int)

{'h': 0, 'm': 1, ' ': 2, 'u': 3, 'e': 4, 'a': 5, 'y': 6, 'r': 7, 'g': 8, 'w': 9, 'd': 10, 'f': 11, 'v': 12, 'c': 13, 'n': 14, 'i': 15, 'o': 16} dict_items([(0, 'h'), (1, 'm'), (2, ' '), (3, 'u'), (4, 'e'), (5, 'a'), (6, 'y'), (7, 'r'), (8, 'g'), (9, 'w'), (10, 'd'), (11, 'f'), (12, 'v'), (13, 'c'), (14, 'n'), (15, 'i'), (16, 'o')])


In [6]:
max(text, key=len) # Gives the longest sentence in the list

'hey how are you'

In [7]:
# Padding

# Adds a ' ' whitespace until the length of the sentence matches the length of the longest sentence

maxlen = len(max(text, key=len))
print("The longest string has {} characters".format(maxlen))

The longest string has 15 characters


In [8]:
for i in range(len(text)):
    while len(text[i])<maxlen:
        text[i] += ' ' # padding spaces at the end of each sentence if required

To predict the next character in the sequence at each time step, divide each sentence into

- Input data
    - The last input character should be excluded as it does not need to be fed into the model
- Target/Ground Truth Label
    - One time-step ahead of the Input data as this will be the "correct answer" for the model at each time step corresponding to the input data

In [9]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
    input_seq.append(text[i][:-1])

    # Remove firsts character for target sequence
    target_seq.append(text[i][1:])
    print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: hey how are yo
Target Sequence: ey how are you
Input Sequence: good i am fine
Target Sequence: ood i am fine 
Input Sequence: have a nice da
Target Sequence: ave a nice day


Convert input and target sequences to sequences of integers instead of characters by mapping them using the dictionaries we created above.

This will allow to one-hot-encode input sequence subsequently.

In [10]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: [0, 5, 12, 4, 2, 5, 2, 14, 15, 13, 4, 2, 10, 5]
Target Sequence: [5, 12, 4, 2, 5, 2, 14, 15, 13, 4, 2, 10, 5, 6]


Before encoding input sequence into one-hot vectors, define 3 key variables:

- *dict_size*: The number of unique characters that we have in our text
    - This will determine the one-hot vector size as each character will have an assigned index in that vector
- *seq_len*: The length of the sequences that we're feeding into the model
    - As we standardised the length of all our sentences to be equal to the longest sentences, this value will be the max length - 1 as we removed the last character input as well
- *batch_size*: The number of sentences that we defined and are going to feed into the model as a batch

In [12]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)

    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

Defined a helper function that creates arrays of zeros for each character and replaces the corresponding character index with a **1**.

In [13]:
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

Input shape: (3, 14, 17) --> (Batch Size, Sequence Length, One-Hot Encoding Size)


After data pre-processing, move the data from numpy arrays to PyTorch's own data structure - **Torch Tensors**

In [14]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

Now define the model using the Torch library.
Add or remove layers like fully connected , convolutational layers, vanilla RNN layers, LSTM layers, and many more!

In this post, we'll use the basic nn.rnn to demonstrate a simple example of how RNNs can be used.

First check if the device is running on (CPU or GPU).
This implementation will not require GPU as the training is really simple. However, for large datasets and models with millions of trainable parameters, using the GPU will be very important to speed up training.

In [15]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


To start the neural network model, define a class that inherits PyTorch’s base class (nn.module) for all neural network modules.

After that, start defining some variables and also the layers for model under the constructor.

For this model, we will use 1 layer of RNN followed by a fully connected layer. The fully connected layer will be in-charge of converting the RNN output to our desired output shape.

Than define the forward pass function under forward() as a class method. The order the forward function is sequentially executed, therefore we first pass the inputs and the zero-initialized hidden state through the RNN layer first, before passing the RNN outputs to the fully-connected layer. Note that we are using the layers that we defined in the constructor.

The last method that we have to define is the method that we called earlier to initialize the hidden state - init_hidden(). This basically creates a tensor of zeros in the shape of our hidden states.

In [16]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)



    def forward(self, x):

        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)

        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

After defining the model above, we'll have to instantiate the model with the relevant parameters and define our hyperparamters as well. The hyperparameters we're defining below are:

- *n_epochs*: Number of Epochs --> This refers to the number of times our model will go through the entire training dataset
- *lr*: Learning Rate --> This affects the rate at which our model updates the weights in the cells each time backpropogation is done
    - A smaller learning rate means that the model changes the values of the weight with a smaller magnitude
    - A larger learning rate means that the weights are updated to a larger extent for each time step

Similar to other neural networks, we have to define the optimizer and loss function as well. We’ll be using CrossEntropyLoss as the final output is basically a classification task.

In [17]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 100
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Now we can begin our training! As we only have a few sentences, this training process is very fast. However, as we progress, larger datasets and deeper models mean that the input data is much larger and the number of parameters within the model that we have to compute is much more.

In [31]:
# Training Run
input_seq = input_seq.to(device)  # Move the input sequence to the specified device (CPU or GPU)

for epoch in range(1, n_epochs + 1):  # Loop over the number of epochs
    optimizer.zero_grad()  # Clears existing gradients from the previous epoch to prevent accumulation

    # Forward pass: compute the output and hidden state of the model
    output, hidden = model(input_seq)

    # Move the output to the specified device (CPU or GPU)
    output = output.to(device)

    # Move the target sequence to the specified device (CPU or GPU)
    target_seq = target_seq.to(device)

    # Compute the loss between the model output and the target sequence
    # The target sequence is reshaped to match the dimensions expected by the loss function
    loss = criterion(output, target_seq.view(-1).long())

    loss.backward()  # Perform backpropagation to compute gradients of the loss with respect to model parameters

    optimizer.step()  # Update the model parameters using the computed gradients

    if epoch % 10 == 0:  # Every 10 epochs, print the current epoch number and loss
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))  # Print the loss formatted to 4 decimal places

Epoch: 10/100............. Loss: 0.1348
Epoch: 20/100............. Loss: 0.1113
Epoch: 30/100............. Loss: 0.0959
Epoch: 40/100............. Loss: 0.0852
Epoch: 50/100............. Loss: 0.0775
Epoch: 60/100............. Loss: 0.0716
Epoch: 70/100............. Loss: 0.0669
Epoch: 80/100............. Loss: 0.0632
Epoch: 90/100............. Loss: 0.0602
Epoch: 100/100............. Loss: 0.0576


Let’s test our model now and see what kind of output we will get. Before that, let’s define some helper function to convert our model output back to text.

In [32]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    # Convert the input character sequence to its corresponding integer representation using char2int dictionary
    character = np.array([[char2int[c] for c in character]])

    # One-hot encode the integer representation to create an input tensor suitable for the model
    character = one_hot_encode(character, dict_size, character.shape[1], 1)

    # Convert the numpy array to a PyTorch tensor
    character = torch.from_numpy(character)

    # Move the tensor to the specified device (CPU or GPU)
    character = character.to(device)

    # Pass the input tensor through the model to get the output and the hidden state
    out, hidden = model(character)

    # Apply the softmax function to the output to get the probabilities of each character
    prob = nn.functional.softmax(out[-1], dim=0).data

    # Find the index of the character with the highest probability
    char_ind = torch.max(prob, dim=0)[1].item()

    # Return the character corresponding to the highest probability index and the hidden state
    return int2char[char_ind], hidden


In [33]:
def sample(model, out_len, start='hey'):
    # Set the model to evaluation mode (disables dropout, batch norm, etc.)
    model.eval()

    # Convert the starting string to lowercase
    start = start.lower()

    # Create a list of characters from the starting string
    chars = [ch for ch in start]

    # Calculate how many more characters we need to generate
    size = out_len - len(chars)

    # Generate characters until we reach the desired output length
    for ii in range(size):
        # Use the predict function to get the next character and the hidden state
        char, h = predict(model, chars)

        # Append the predicted character to the list
        chars.append(char)

    # Join the list of characters into a single string and return it
    return ''.join(chars)


In [26]:
sample(model, 20, 'hey')

'hey how are youe are'

As we can see, the model is able to come up with the sentence ‘good i am fine ‘ if we feed it with the words ‘good’, achieving what we intended for it to do!

In [27]:
torch.save(model.state_dict(), 'checkpoint.pth')

In [28]:
# prompt: load checkpoint from checkpoint.pth into model1 variable

model1 = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
model1.load_state_dict(torch.load('checkpoint.pth'))
model1.eval()


Model(
  (rnn): RNN(17, 12, batch_first=True)
  (fc): Linear(in_features=12, out_features=17, bias=True)
)

In [29]:
def sample1(model, out_len, start='hey'):
    model1.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model1, chars)
        chars.append(char)

    return ''.join(chars)

In [30]:
sample1(model1, 15, 'good')

'good i am fine '